### IO and utilities

In [84]:
import os
from dataclasses import dataclass, fields
import re

def read_file(path: str) -> list[str]:
    with open(os.path.join(os.getcwd(), path), "r") as f:
        return [line.strip() for line in f.readlines()]
    
StringMatrix = list[str]

def read_file_as_matrix(path: str) -> StringMatrix:
    with open(os.path.join(os.getcwd(), path), "r") as f:
        return [line.strip() for line in f.readlines()]


# Day 1: Trebuchet?!


In [85]:
def getnumber(line: str) -> int:
    numbers = [c for c in line if c.isdigit()]
    return int(numbers[0] + numbers[-1])

day1_1 = read_file("day1/example.txt")
print(sum([getnumber(l) for l in day1_1]))


142


Part 2, search each line for the first digit or word, the search from back after first digit or word.

In [86]:
def test(substr: str) -> str:
    numbers = {
        'one': '1',
        'two': '2',
        'three': '3',
        'four': '4',
        'five': '5',
        'six': '6',
        'seven': '7',
        'eight': '8',
        'nine': '9'
    }

    for word, digit in numbers.items():
        if word in substr:
            return digit

    return ''

def find_first_digit(inline: str, reverse = False) -> str:
    buffer = ''
    line = inline[::-1] if reverse else inline

    for c in line:
        if c.isdigit():
            return c

        if reverse:
            buffer = c + buffer
        else:
            buffer = buffer + c

        candidate = test(buffer)
        if candidate:
            return candidate

def replace_text(line: str) -> str:
    return find_first_digit(line) + find_first_digit(line, True)
   
day1_2 = read_file("day1/example.txt")
parsed = [int(replace_text(l)) for l in day1_2]
print(sum(parsed))

142


# Day 2: Cube Conundrum

### Data representation and parsing

In [87]:
@dataclass
class GameSet():
    red: int
    green: int
    blue: int

@dataclass
class Game():
    id: int
    sets: list[GameSet]

def parse_set(s: str) -> GameSet:
    gameset = GameSet(0,0,0)
    cubesstr = s.split(',')

    for cube in cubesstr:
        n = int(re.search('\d+', cube).group())
        if 'red' in cube:
            gameset.red = n
        elif 'green' in cube:
            gameset.green = n
        else:
            gameset.blue = n
    
    return gameset


def parse(line: str) -> Game:
    g = Game(-1, [])
    
    gamesrt, setsstr = line.split(':')
    g.id = int(gamesrt[4:])
    g.sets = [parse_set(s) for s in setsstr.split(';')]
    return g

day2_1 = read_file('day2/example.txt')
games = [parse(l) for l in day2_1]

### Part 1: Finding possible games

In [88]:

def max_set(a: GameSet, b: GameSet) -> GameSet:
    return GameSet(max(a.red, b.red), max(a.green, b.green), max(a.blue, b.blue))

def possible(game: Game, max_allowed: GameSet) -> bool:
    game_max = GameSet(0,0,0)

    for gameset in game.sets:
        game_max = max_set(game_max, gameset)

    return game_max.red <= max_allowed.red and game_max.green <= max_allowed.green and game_max.blue <= max_allowed.blue

valid_game_ids = [g.id for g in games if possible(g, GameSet(12, 13, 14))]
print(sum(valid_game_ids))


8


### Part 2: Finding mimimum possible cubes and summing power of these sets

In [89]:
def minimum_possible_cubes(game: Game) -> GameSet:
    game_min = GameSet(0,0,0)

    for gameset in game.sets:
        game_min = max_set(game_min, gameset)

    return game_min

def game_power(game: Game) -> int:
    s = minimum_possible_cubes(game)
    return s.red * s.green * s.blue

print(sum([game_power(g) for g in games]))

2286


# Day 3: Gear Ratios

### Utils

In [90]:

@dataclass
class Coord():
    row: int
    col: int

def getAdjecent(coord: Coord, total_rows: int, total_cols: int) -> list[Coord] :
    result = []
    
    neighbouring_left = coord.col > 0
    neighbouring_right = coord.col < total_cols
    neighbouring_up = coord.row > 0
    neighbouring_down = coord.row < total_rows

    if neighbouring_up:
        if neighbouring_left:
            result.append(Coord(coord.row - 1, coord.col - 1))
        
        result.append(Coord(coord.row - 1, coord.col))

        if neighbouring_right:
            result.append(Coord(coord.row - 1, coord.col + 1))
    
    if neighbouring_left:
        result.append(Coord(coord.row, coord.col - 1))

    if neighbouring_right:
        result.append(Coord(coord.row, coord.col + 1))

    if neighbouring_down:
        if neighbouring_left:
            result.append(Coord(coord.row + 1, coord.col - 1))
        
        result.append(Coord(coord.row + 1, coord.col))

        if neighbouring_right:
            result.append(Coord(coord.row + 1, coord.col + 1))

    return result

def get_char(data: StringMatrix, coord: Coord) -> str:
    return data[coord.row][coord.col]

def is_digit(data: StringMatrix, coord: Coord) -> bool:
    return get_char(data, coord).isdigit()

def getIntIndex(data: StringMatrix, coord: Coord) -> (int,int):
    found_first = False
    found_last = False 

    f_index = coord.col
    l_index = coord.col

    while not found_first:
        candidate = f_index - 1
        if candidate >= 0 and data[coord.row][candidate].isdigit():
            f_index = candidate
        else:
            found_first = True

    while not found_last:
        candidate = l_index + 1
        if candidate < len(data[coord.row]) and data[coord.row][candidate].isdigit():
            l_index = candidate
        else:
            found_last = True

    return f_index, l_index

def remove_data(data: StringMatrix, row, begin, end):
    current = data[row]
    data[row] = current[:begin] + 'x' * (end + 1 - begin) + current[end+1:]



### Part 1

In [91]:
def is_symbol(data: StringMatrix, coord: Coord) -> bool:
    symbols = ["+","*","#","$","%","@","#","&","=","-",'/']
    return get_char(data, coord) in symbols

result = 0
data = read_file_as_matrix("day3/example.txt")
for row in range(len(data)):
    for col in range(len(data[row])):
        c = Coord(row, col)
        if is_symbol(data, c):
            adjecents = getAdjecent(c, len(data), len(data[row]))

            for a in adjecents:
                if is_digit(data, a):
                    first, last = getIntIndex(data, a)
                    number = int(data[a.row][first:last+1])
                    result += number
                    remove_data(data, a.row, first, last)

print(result)

4361


### Part 2 - find gears

In [92]:
def is_gear(data: StringMatrix, coord: Coord) -> bool:
    return get_char(data, coord) == '*'

result = 0
data = read_file_as_matrix("day3/example.txt")
for row in range(len(data)):
    for col in range(len(data[row])):
        c = Coord(row, col)
        if is_gear(data, c):
            adjecents = getAdjecent(c, len(data), len(data[row]))
            
            numbers = []

            data_before = data

            for a in adjecents:
                if is_digit(data, a):
                    first, last = getIntIndex(data, a)
                    number = int(data[a.row][first:last+1])
                    numbers.append(number)
                    remove_data(data, a.row, first, last)

            if len(numbers) == 2:
                result += numbers[0] * numbers[1]
            else: 
                # No gear! Restore data
                data = data_before
print(result)

467835


# Day 4: Scratchcards

### Part 1

In [93]:
@dataclass
class Card():
    id: int
    winning_numbers: set[int]
    numbers: set[int]
    
def parse_card(line: str) -> Card:
    card_nr_str, rest = line.split(":")
    c = Card(int(card_nr_str[5:].strip()), [], [])

    winning_str, numbers_str = rest.split("|")
    c.winning_numbers = {int(n.strip()) for n in winning_str.split()}
    c.numbers = {int(n.strip()) for n in numbers_str.split()}

    return c

def worth(c: Card) -> int:
    wins = c.numbers.intersection(c.winning_numbers)
    return 0 if len(wins) == 0 else pow(2, len(wins) - 1)

data = read_file("day4/example.txt")
cards = [parse_card(l) for l in data]
winnings = [worth(c) for c in cards]
print(sum(winnings))


13


### Part 2

In [94]:
@dataclass
class Card():
    id: int
    winning_numbers: set[int]
    numbers: set[int]
    copies: int
    
def parse_card(line: str) -> Card:
    card_nr_str, rest = line.split(":")
    c = Card(int(card_nr_str[5:].strip()), [], [], 1)

    winning_str, numbers_str = rest.split("|")
    c.winning_numbers = {int(n.strip()) for n in winning_str.split()}
    c.numbers = {int(n.strip()) for n in numbers_str.split()}

    return c

def wins(c: Card) -> int:
    wins = len(c.numbers.intersection(c.winning_numbers))
    return range(c.id, c.id+wins)
      

data = read_file("day4/example.txt")
cards = [parse_card(l) for l in data]
for card in cards:
    new_copies = wins(card)
    for id in new_copies:
        if id < len(cards):
            cards[id].copies += card.copies

total = sum([c.copies for c in cards])
print(total)

30


# Day 5: If You Give A Seed A Fertilizer

### Utils - shared code between Part 1 and Part 2

In [95]:
import sys
import operator

@dataclass
class Table_item():
    destination_range_start: int
    source_range_start: int
    range_length: int

def within (src: int, entry: Table_item) -> int:
    src_delta = src - entry.source_range_start
    if src_delta >= 0 and src_delta < entry.range_length:
        return entry.destination_range_start + src_delta
    return -1

def parse_tables(lines: list[str]) -> dict[str, list[Table_item]]:
    tables = {}

    parsing_map = False    
    current_map = ''

    for line in lines:
        if len(line) == 0:
            parsing_map = False
            if current_map:
                current_map = ''
            continue
        if not parsing_map and ' map:' in line:
            current_map = line.split(" map:")[0]
            parsing_map = True
            continue

        if parsing_map:
            d, s, r = [int(x) for x in line.split()]
            
            if not current_map in tables:
                tables[current_map] = []
            
            tables[current_map].append(Table_item(d, s, r))
    
    for table in tables.values():
        table.sort(key=operator.attrgetter('source_range_start'))

    return tables

def populate_seeds(seed_ids: list[int], tables: dict[str, list[Table_item]]) -> list[list[int]]:
    seeds = [[id] + [-1] * 7 for id in seed_ids]
    keys = ['seed-to-soil', 'soil-to-fertilizer', 'fertilizer-to-water', 'water-to-light', 'light-to-temperature', 'temperature-to-humidity', 'humidity-to-location']
    
    for seed in seeds:
        for i, key in enumerate(keys, 1):
            src = i-1
            dst = i
            
            seed[dst] = seed[src] # Default to prev value

            for entry in tables[key]:
                v = within(seed[src], entry)
                if v > -1:
                    seed[dst] = v
                    break
    
    return seeds

def find_lowest_location(seeds: list[list[int]]) -> int:
    lowest_location = sys.maxsize

    for seed in seeds:
        lowest_location = min(lowest_location, seed[7]) # Location at index 7

    return lowest_location


### Part 1

In [96]:
def parse_seeds(line: str) -> list[int]:
    return [int(x) for x in line[7:].split()]

data = read_file("day5/example.txt")
seed_ids = parse_seeds(data[0])
tables = parse_tables(data[1:])
seeds = populate_seeds(seed_ids, tables)
print(find_lowest_location(seeds))

51752125


### Part 2
Needed to solve it based on seed ranges instead of simulating every seed. The solution from part 1 when applied to part 2 was too slow

In [97]:
@dataclass
class Number_Range():
    start: int
    length: int

def parse_seeds(line: str) -> list[Number_Range]:
    seed_numbers = [int(x) for x in line[7:].split()]
    
    seeds = []
    for i in range(0, len(seed_numbers), 2):
        seeds.append(Number_Range(seed_numbers[i], seed_numbers[i+1]))

    seeds.sort(key=operator.attrgetter('start'))
    return seeds

def convert_range(range: Number_Range, entry: Table_item) -> (Number_Range | None, Number_Range | None, Number_Range | None):
    range_end = range.start + range.length
    entry_end = entry.source_range_start + entry.range_length

    if range_end < entry.source_range_start:
        below = range
        return below, None, None
    
    if entry_end <= range.start:
        above = range
        return None, None, above

    # all within
    if range.start >= entry.source_range_start and range_end <= entry_end:
        start_delta = range.start - entry.source_range_start
        converted = Number_Range(entry.destination_range_start + start_delta, range.length)
        return None, converted, None
    
    if range.start < entry.source_range_start and range_end <= entry_end:
        delta = entry.source_range_start - range.start
        below = Number_Range(range.start, delta)
        converted = Number_Range(entry.destination_range_start, range.length - delta)
        return below, converted, None

    if range.start >= entry.source_range_start and range_end > entry_end:
        delta = range_end - entry_end
        above = Number_Range(entry_end, delta)
        delta_start = range.start - entry.source_range_start
        converted = Number_Range(entry.destination_range_start + delta_start, range.length - delta)
        return None, converted, above

    if range.start < entry.source_range_start and range_end > entry_end:
        delta_b = entry.source_range_start - range.start
        below = Number_Range(range.start, delta_b)

        delta_a = range_end - entry_end
        above = Number_Range(entry_end, delta_a)
        
        converted = Number_Range(entry.destination_range_start, entry.range_length)
        return below, converted, above
    
def lookup(range: Number_Range, table: list[Table_item]) -> list[Number_Range]:
    # We assume that table is sorted based on source start

    remaining = range
    processed = []
    for entry in table:
        below, converted, above = convert_range(remaining, entry)

        if below:
            processed.append(below)

        if converted:
            processed.append(converted)
        
        remaining = above

        if not remaining:
            break

    if remaining:
        processed.append(remaining)
    
    return processed

def solve(seeds: list[Number_Range], tables: dict[str, list[Table_item]]) -> int:
    table_names = ['seed-to-soil', 'soil-to-fertilizer', 'fertilizer-to-water', 'water-to-light', 'light-to-temperature', 'temperature-to-humidity', 'humidity-to-location']
    
    unprocessed_range = seeds
    proccessed = []

    for name in table_names:
        proccessed = []
        for nr in unprocessed_range:
            proccessed.extend(lookup(nr, tables[name]))

        unprocessed_range = proccessed
    
    min_location = sys.maxsize
    for x in proccessed:
        min_location = min(min_location, x.start)

    return min_location

data = read_file("day5/example.txt")
seed_ranges = parse_seeds(data[0])
tables = parse_tables(data[1:])

lowest_location = solve(seed_ranges, tables)
print(lowest_location)

12634632
